In [1]:
import numpy as np
import pandas as pd


def compute_composite_score(df):
    df['time_taken_norm'] = df.groupby('instance_id')['time_taken'].transform(
        lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8))
    df['distance_gap_norm'] = df.groupby('instance_id')['distance_gap'].transform(
        lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8))

    is_small = df['nr_cities'] <= 40
    is_large = df['nr_cities'] > 40

    strat = df['strategy'].str.lower()

    # penalize strategies based on their characteristics
    strat_bonus = np.where(strat == 'greedy', -0.02, 
                   np.where(strat == 'kmeans greedy', 0.0,
                   np.where(strat == 'branch and cut', np.where(is_large, 0.2, -0.3),
                   np.where(strat == 'ant colony', -0.1, 0.0))))

    df['composite_score'] = (
        df['normalized_cost'] * 0.65 +
        df['time_taken_norm'] * 0.25 +
        df['distance_gap_norm'] * 0.1 +
        strat_bonus
    )
    return df

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import sqlite3

conn = sqlite3.connect('../train_mTSP.sqlite3')
instances = pd.read_sql_query("SELECT * FROM instances", conn)
algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)
conn.close()

data = pd.merge(instances, algorithms, on="instance_id")

# Filter out branch and cut for large instances (they tend to give bad distributions)
data = data[~((data['strategy'] == 'branch and cut') & (data['nr_cities'] > 50))]

# Compute composite score 

data = compute_composite_score(data)

# Select best strategy per instance
best_strategies = data.loc[data.groupby('instance_id')['composite_score'].idxmin()]

# Select features and labels
features = best_strategies[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                            'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                            'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]
labels = pd.get_dummies(best_strategies['strategy'])
labels_order = list(labels.columns)
with open('labels_order.txt', 'w') as f:
    for col in labels_order:
        f.write(col + '\n')

# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)

x_train = torch.tensor(features, dtype=torch.float32)
y_train = torch.tensor(labels.values, dtype=torch.float32)


conn = sqlite3.connect('../validation_mTSP.sqlite3')
validation_instances = pd.read_sql_query("SELECT * FROM instances", conn)
validation_algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)
conn.close()

validation_data = pd.merge(validation_instances, validation_algorithms, on="instance_id")
validation_data = validation_data[~((validation_data['strategy'] == 'branch and cut') & (validation_data['nr_cities'] > 50))]

# Compute composite score for validation data
validation_data = compute_composite_score(validation_data)

# Select best strategy per validation instance
validation_best = validation_data.loc[validation_data.groupby('instance_id')['composite_score'].idxmin()]

# Select features and labels for validation
validation_features = validation_best[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                                       'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                                       'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]
validation_labels = pd.get_dummies(validation_best['strategy'])

# ensure validation labels have the same columns as training labels
for col in labels.columns:
    if col not in validation_labels.columns:
        validation_labels[col] = 0
validation_labels = validation_labels[labels.columns]

# Normalize validation features
validation_features = scaler.transform(validation_features)

# Ensure validation labels have the same columns as training labels
for col in labels.columns:
    if col not in validation_labels.columns:
        validation_labels[col] = 0
validation_labels = validation_labels[labels.columns]  # ensure same order
validation_labels = validation_labels.astype(float)

x_val = torch.tensor(validation_features, dtype=torch.float32)
y_val = torch.tensor(validation_labels.values, dtype=torch.float32)

# Neural Network Model 
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(32, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return self.softmax(x)

input_size = x_train.shape[1]
output_size = y_train.shape[1]
model = NeuralNetwork(input_size, output_size)

# Calculate class weights for imbalanced classes so that the model pays more attention to underrepresented classes
class_counts = best_strategies['strategy'].value_counts()
class_weights = 1.0 / class_counts
class_weights_dict = {strategy: class_weights[strategy] for strategy in class_counts.index}
weights = torch.tensor([class_weights_dict[label] for label in labels.columns], dtype=torch.float32)
criterion = nn.CrossEntropyLoss(weight=weights)
# criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

best_val_loss = float('inf')
patience = 25  # number of epochs to wait for improvement
trigger_times = 0

# Train
epochs = 300
batch_size = 32
for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(x_train.size(0))
    for i in range(0, x_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_X, batch_y = x_train[indices], y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, torch.argmax(batch_y, dim=1))
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_val)
        val_loss = criterion(val_outputs, torch.argmax(y_val, dim=1))
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    # Stop if no improvement in validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        torch.save(model.state_dict(), 'models/best_model.pth')  
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping triggered!")
            break

    scheduler.step()

# torch.save(model.state_dict(), 'algorithm_selector_model2.pth')


Epoch 1/300, Train Loss: 1.0049, Val Loss: 1.0349
Epoch 2/300, Train Loss: 0.9363, Val Loss: 0.9584
Epoch 3/300, Train Loss: 0.7790, Val Loss: 0.8810
Epoch 4/300, Train Loss: 0.5834, Val Loss: 0.8250
Epoch 5/300, Train Loss: 0.5665, Val Loss: 0.8141
Epoch 6/300, Train Loss: 0.5515, Val Loss: 0.7966
Epoch 7/300, Train Loss: 0.6648, Val Loss: 0.7868
Epoch 8/300, Train Loss: 0.5532, Val Loss: 0.7826
Epoch 9/300, Train Loss: 1.1890, Val Loss: 0.7729
Epoch 10/300, Train Loss: 0.7666, Val Loss: 0.7682
Epoch 11/300, Train Loss: 1.3765, Val Loss: 0.7686
Epoch 12/300, Train Loss: 1.3908, Val Loss: 0.7620
Epoch 13/300, Train Loss: 1.3711, Val Loss: 0.7596
Epoch 14/300, Train Loss: 0.5529, Val Loss: 0.7560
Epoch 15/300, Train Loss: 1.2254, Val Loss: 0.7578
Epoch 16/300, Train Loss: 0.5792, Val Loss: 0.7587
Epoch 17/300, Train Loss: 0.7327, Val Loss: 0.7563
Epoch 18/300, Train Loss: 0.5608, Val Loss: 0.7576
Epoch 19/300, Train Loss: 1.3858, Val Loss: 0.7570
Epoch 20/300, Train Loss: 0.5514, Val Lo

In [45]:
print(best_strategies['strategy'].value_counts())

with torch.no_grad():
    train_outputs = model(x_train)
    train_predictions = torch.argmax(train_outputs, dim=1)
    print(pd.Series(train_predictions.numpy()).value_counts())

strategy
Branch and Cut    661
Greedy            367
KMeans-Greedy      59
Ant Colony          2
Name: count, dtype: int64
1    647
2    262
3    171
0      9
Name: count, dtype: int64
